In [16]:
!pip install python-constraint

In [17]:
from constraint import Problem
from itertools import combinations
import requests

def get_subsets(transactions, min_support):
    itemsets = {}
    for transaction in transactions:
        for i in range(1, len(transaction) + 1):
            for subset in combinations(transaction, i):
                subset = frozenset(subset)
                if subset in itemsets:
                    itemsets[subset] += 1
                else:
                    itemsets[subset] = 1
    frequent_itemsets = {k: v for k, v in itemsets.items() if v >= min_support}
    return frequent_itemsets

# association rule
def get_association_rules(frequent_itemsets, min_confidence, transactions):
    rules = []
    for itemset in frequent_itemsets:
        if len(itemset) > 1:
            for antecedent in combinations(itemset, len(itemset) - 1):
                antecedent = frozenset(antecedent)
                consequent = itemset - antecedent
                antecedent_support = sum(1 for transaction in transactions if antecedent <= transaction)
                confidence = frequent_itemsets[itemset] / antecedent_support
                if confidence >= min_confidence:
                    rules.append((antecedent, consequent, confidence))
    return rules


def read_transactions_from_url(url):
    response = requests.get(url)
    response.raise_for_status()
    lines = response.text.strip().split('\n')
    transactions = [line.strip().split(',') for line in lines]
    return [frozenset(transaction) for transaction in transactions]


if __name__ == "__main__":
    url = 'https://raw.githubusercontent.com/PSCostaM/TA3_CC82_TopicsCs/master/transactions.txt'
    transactions = read_transactions_from_url(url)
    min_support = 2
    min_confidence = 0.6

    frequent_itemsets = get_subsets(transactions, min_support)
    print("Itemsets frecuentes:", frequent_itemsets)

    association_rules = get_association_rules(frequent_itemsets, min_confidence, transactions)
    print("Reglas de asociación:")
    for rule in association_rules:
        print(f"{set(rule[0])} -> {set(rule[1])} (confianza: {rule[2]:.2f})")


Itemsets frecuentes: {frozenset({'leche'}): 5, frozenset({'mantequilla'}): 4, frozenset({'pan'}): 6, frozenset({'leche', 'mantequilla'}): 3, frozenset({'leche', 'pan'}): 4, frozenset({'mantequilla', 'pan'}): 3, frozenset({'leche', 'mantequilla', 'pan'}): 2}
Reglas de asociación:
{'leche'} -> {'mantequilla'} (confianza: 0.60)
{'mantequilla'} -> {'leche'} (confianza: 0.75)
{'leche'} -> {'pan'} (confianza: 0.80)
{'pan'} -> {'leche'} (confianza: 0.67)
{'mantequilla'} -> {'pan'} (confianza: 0.75)
{'leche', 'mantequilla'} -> {'pan'} (confianza: 0.67)
{'mantequilla', 'pan'} -> {'leche'} (confianza: 0.67)
